In [ ]:
import math
import numpy as np

In [ ]:

def simulate_paths(M, N, r, vol, S, K, dt):  #same problem here
    def pay_off(n, S):
        return np.exp(-r * n * dt) * np.maximum(0, K - S)

    nudt = (r - 0.5 * vol**2) * dt
    volsdt = vol * np.sqrt(dt)
    lnS = np.log(S)

    # Méthode de Monte Carlo
    Z = np.random.normal(size=(M, N))
    delta_lnSt = nudt + volsdt * Z
    delta_lnSt_anti = nudt - volsdt * Z

    LnS_s = np.zeros([M, N + 1])
    LnS_s_anti= np.zeros([M, N + 1])


    # Set the initial values
    LnS_s[:, 0] = lnS
    LnS_s_anti[:, 0] = lnS

    # Compute cumulative sums using cumsum
    LnS_s[:, 1:] = np.cumsum(delta_lnSt, axis=1)
    LnS_s[:,1:] +=lnS

    LnS_s_anti[:, 1:] = np.cumsum(delta_lnSt_anti, axis=1)
    LnS_s_anti[:,1:] +=lnS

    X = np.vstack((np.exp(LnS_s),np.exp(LnS_s_anti)))
    P = np.zeros_like(X)
    for n in range(N + 1):
        P[:, n] = pay_off(n, X[:, n])
    return X, P

S = 36          # Prix de l'action
K = 40          # Prix d'exercice
vol = 0.2        # Volatilité (%)
r = 0.06            # Taux sans risque (%)
                     #Pour Delta t

M=100
Tf = 1 #date d'echeance
N=50 #nombre de point d'execution avec T0=0 et TN=Tf.
dt=Tf/N
T = np.arange(0,N+1,1) #contient T0,T1,T2,....Tf.





X,Z=simulate_paths(M, N, r, vol, S, K, dt)
Z0=Z[0,0]


In [ ]:
def laguerre_basis(x, n):
      # Constant term
        return np.exp(-x / 2) * np.polynomial.Laguerre([0] * n + [1])(x)


In [ ]:

from sklearn.linear_model import LinearRegression

#Here where we save the discounted value of the bermudan option for each path
U=np.zeros([2*M,1]) #M
U=np.copy(Z[:,N].reshape(-1,1) ) #N is equivalent to say it's TN

# Create the design matrix with Laguerre polynomials
degree = 5
# Regression of U[in_money_indexes] w.r.t. paths_needed using Laguerre polynomials
model = LinearRegression()

for t in range(N-1, 0, -1):
    in_money_indexes = np.where(Z[:, t] > 0)[0]
    N_paths = len(in_money_indexes)
    paths_needed = X[in_money_indexes, t]


    # Create the design matrix with Laguerre polynomials and a column of ones
    if len(paths_needed) > 0:
      # Create the design matrix with Laguerre polynomials and a column of ones
      design_matrix = np.column_stack([laguerre_basis(paths_needed, i).reshape(-1, 1) for i in range(degree)])

      # Add a column of ones
      design_matrix = np.column_stack([np.ones(len(paths_needed)).reshape(-1,1),design_matrix])

      model.fit(design_matrix, U[in_money_indexes])


      # reg = np.polyfit(paths_needed,U[in_money_indexes], 7)
      # C_t = np.polyval(reg, paths_needed)

      # Predict the continuation values
      C_t = model.predict(design_matrix)

      # Update U for paths where C_t < Z[in_money_indexes, t]
      U[in_money_indexes] = np.where(C_t.reshape(-1, 1) > Z[in_money_indexes, t].reshape(-1, 1), U[in_money_indexes], Z[in_money_indexes, t].reshape(-1, 1))




# Calculate the approximate price at time 0 of the option
U0 = np.mean(U)


print(f"Approximate option price at time 0: {U0}")
#I think the error is caused by the discount that I did,


Approximate option price at time 0: 4.468215283435618


In our regression problem we used as a  basis functions is the set of
Laguerre polynomials until degree 5.

And with 50,000 plus 50,000 antithetic simulated path of the underlying asset price.
With the identical parameters of the article [4] (page 127 , first row ) the american option approximate price we got is  4.468215283435618 , that we compare with the results of article [4] which are 4.472 .

The results we obtained are very close to those reported in the article.

